In [59]:
from docx_parser import DocumentParser
import numpy as np
infile = 'sample2.docx'
doc = DocumentParser(infile)
arr = np.array([])
for _type, item in doc.parse():
    arr = np.append(arr, item)
#print(arr[0]['text'])
print(arr)

[{'text': 'Quantum Aristoxeni ingenium consumptum videmus in musicis?', 'style_id': 'a'}
 {'text': 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quid nunc honeste dicit? Tum Torquatus: Prorsus, inquit, assentior; Duo Reges: constructio interrete. Iam in altera philosophiae parte. Sed haec omittamus; Haec para/doca illi, nos admirabilia dicamus. Nihil sane.', 'style_id': 'a'}
 {'text': 'Expressa vero in iis aetatibus, quae iam confirmatae sunt.', 'style_id': 'a'}
 {'text': 'Sit sane ista voluptas. Non quam nostram quidem, inquit Pomponius iocans; An tu me de L. Sed haec omittamus; Cave putes quicquam esse verius.', 'style_id': 'a'}
 {'data': [['sobaka', 'sobaka', 'sobaka', 'sobaka', '', '', 'dawn'], ['1', '2', '3', '', '', '', 'dawn'], ['6', '5', '4', '', '', '', 'dawn'], ['', '', '', '', '', '', 'dawn'], ['', '', '', '121', '', '', 'dawn']], 'merged_cells': {'0_1_0_4': 'sobaka', '0_5_6_7': 'dawn'}}
 {'text': 'KEKS', 'style_id': 'a5'} {'text': 'Wddw', 'style_id': 'a5'}
 {'te